## 1.计算品退率

In [11]:
import pandas as pd
import numpy as np
path_rate="/Users/philyu/kim/Kim file/2021-02/02品退数据/0131_品质退款率_各类目.csv"
#path_rate="/Users/philyu/kim/Kim file/2021-01/品退数据/1221_各级类目-品质退款率.csv"
data=pd.read_csv(path_rate)
data.head()

,p_date,category_level_1_name,category_level_2_name,category_level_3_name,cps_type,order_count,quality_refund_count,quality_refund_rate
0,20200723,食品,生鲜,蔬菜瓜果,0,3058710.0,37320,0.012201
1,20200723,家用电器,厨卫电器,净水饮水,1,224.0,1,0.004464
2,20200722,手机/数码/电脑办公,电脑/外设,网络产品,0,9953.0,242,0.024314
3,20200722,家用电器,大家电,空调,0,3268.0,69,0.021114
4,20200723,珠宝/钟表,玉石,和田玉,0,773276.0,6732,0.008706


### 1.1 品退率计算

In [12]:
#（1）一级类目品退率
df_1=data.groupby("category_level_1_name").agg(order_count=("order_count", "sum"),quality_refund_count=("quality_refund_count","sum"))
df_1.loc[:,"一级类目品退率"]=df_1["quality_refund_count"]/df_1["order_count"]
df_1=df_1.reset_index()
#（2）二级类目品退率
df_2=data.groupby("category_level_2_name").agg(order_count=("order_count", "sum"),quality_refund_count=("quality_refund_count","sum"))
df_2.loc[:,"二级类目品退率"]=df_2["quality_refund_count"]/df_2["order_count"]
df_2=df_2.reset_index()
# （3）三级类目品退率
df_3=data.groupby("category_level_3_name").agg(order_count=("order_count", "sum"),quality_refund_count=("quality_refund_count","sum"))
df_3.loc[:,"三级类目品退率"]=df_3["quality_refund_count"]/df_3["order_count"]
df_3=df_3.reset_index()

## 2.分销库

In [13]:
#path="/Users/philyu/kim/Kim file/2021-01/品退数据/0103_近60天品质退款率-区分分销库商品-退款订单大于5单.csv"
path="/Users/philyu/kim/Kim file/2021-02/02品退数据/0131_近60天品质退款率-区分分销库商品-退款订单大于5单.csv"
data_is=pd.read_csv(path)
condition_is=(data_is['is_cps']==1)&(data_is['on_offline_status']==1)&(data_is['quality_refound_cnt']<=(data_is['order_cnt']))&(data_is['order_cnt']>1000)
data_is=data_is[condition_is]
len(data_is)

1348

In [14]:
data_is_1=data_is.groupby("category_level_3_name")["item_id"].count()
data_is_1_1=pd.DataFrame(data_is_1)
data_is_1_1.columns=['分销库在架商品数'] #修改列索引名称
data_is_1_1=data_is_1_1.reset_index()
#品退率与商品匹配（vlookup）
data_is_2=pd.merge(left=data_is_1_1,right=df_3,left_on="category_level_3_name",right_on="category_level_3_name")
data_is_2.loc[:,"五倍三级类目品退率"]=data_is_2["三级类目品退率"].apply(lambda x:x*5)
#数据合并
data_is_five=data_is_2[["category_level_3_name","五倍三级类目品退率"]]
data_is_all=pd.merge(left=data_is,right=data_is_five,left_on="category_level_3_name",right_on="category_level_3_name",)
data_is_all.head(5)

,p_date,seller_id,seller_name,item_id,source_type,is_cps,item_title,item_link_url,on_offline_status,item_status,...,category_level_3_name,order_cnt,quality_refound_cnt,item_quality_refound_cnt,disdescription_refound_cnt,fake_refound_cnt,miss_refound_cnt,damaged_refound_cnt,quality_refound_rate,五倍三级类目品退率
0,20210131,1870041877,罗原食品,829078565877,自建,1,南北干货桂圆干龙眼干福建特产,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,南北干货,3554,6,0,1,0,1,4,0.001688,0.008345
1,20210131,136785529,海蓝之星白酒,861512204529,自建,1,黑枸杞,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,南北干货,4857,13,4,2,0,4,3,0.002677,0.008345
2,20210131,2159313324,椿林火锅底料厂家直销,702852249324,自建,1,重庆火锅川粉8袋下单送2袋（每袋200g）,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,南北干货,3469,18,1,0,0,4,13,0.005189,0.008345
3,20210131,841105201,老魏心选,812640299201,自建,1,【小萱专属】3斤正宗纯手工红薯粉条火锅粉条,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,南北干货,5265,8,4,3,0,0,1,0.001519,0.008345
4,20210131,1368076544,飘叶公子好物严选,422498197544,自建,1,浮夏农家红薯粉条4斤装纯手工Q,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,南北干货,2234,7,2,0,0,1,4,0.003133,0.008345


In [16]:
data_is_preoff=data_is_all[(data_is_all["quality_refound_rate"]>data_is_all["五倍三级类目品退率"])]
data_is_3=data_is_preoff.groupby("category_level_3_name")["item_id"].count()
data_is_3_1=pd.DataFrame(data_is_3)
data_is_3_1.columns=['分销库待下架商品数'] #修改列索引名称
#三级类目品退率和商品品退情况匹配（vlookup）
data_is_4=pd.merge(left=data_is_2,right=data_is_3_1,left_on="category_level_3_name",right_on="category_level_3_name")#,how='outer'
data_is_print=data_is_4[["category_level_3_name","三级类目品退率","五倍三级类目品退率","分销库待下架商品数","分销库在架商品数"]]

### 分销库数据导出

In [17]:
import pandas as pd
writer = pd.ExcelWriter("/Users/philyu/kim/Kim file/2021-02/02品退数据/0131_分销库品退率情况_分析4.xlsx")
data_is.to_excel(writer, sheet_name='分销库数据',index=None)
data_is_print.to_excel(writer, sheet_name='分销库品退率情况',index=None)
data_is_preoff.to_excel(writer, sheet_name='分销库待下架商品名单',index=None)

writer.save()
writer.close()

/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


## 3.非分销库计算

### 一级类目计算

In [7]:
#含食品数据
data_no1=pd.read_csv(path)
condition_no=(data_no1['is_cps']==0)&(data_no1['on_offline_status']==1)&(data_no1['quality_refound_cnt']<=data_no1['order_cnt'])
data_no1=data_no1[condition_no]
print("非分销库数据个数：",len(data_no1))
#不含食品数据
data_no=pd.read_csv(path)
condition_no=(data_no['is_cps']==0)&(data_no['on_offline_status']==1)&(data_no['quality_refound_cnt']<=data_no['order_cnt'])&(data_no['category_level_1_name']!="食品")
data_no=data_no[condition_no]
print("非分销库不含食品数据个数：",len(data_no))

非分销库数据个数： 5963
非分销库不含食品数据个数： 4786


In [8]:
#非分销库下架规则，十倍一级类目品退率
data_no_1=data_no.groupby("category_level_1_name")["item_id"].count()
data_no_1_1=pd.DataFrame(data_no_1)
data_no_1_1.columns=['非分销库在架商品数'] #修改列索引名称
data_no_1_1=data_no_1_1.reset_index()
data_no_2=pd.merge(left=data_no_1_1,right=df_1,left_on="category_level_1_name",right_on="category_level_1_name")
data_no_2.loc[:,"十倍一级类目品退率"]=data_no_2["一级类目品退率"].apply(lambda x:x*10)
#非分销库品退率与商品匹配
data_no_ten=data_no_2[["category_level_1_name","十倍一级类目品退率"]]
data_no_all=pd.merge(left=data_no,right=data_no_ten,left_on="category_level_1_name",right_on="category_level_1_name",)
data_no_all.head(10)

,p_date,seller_id,seller_name,item_id,source_type,is_cps,item_title,item_link_url,on_offline_status,item_status,...,category_level_3_name,order_cnt,quality_refound_cnt,item_quality_refound_cnt,disdescription_refound_cnt,fake_refound_cnt,miss_refound_cnt,damaged_refound_cnt,quality_refound_rate,十倍一级类目品退率
0,20210131,463959045,二姐品牌女鞋,508588504045,自建,0,精品鞋子4,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,高跟鞋,3644,55,16,7,0,3,29,0.015093,0.028822
1,20210131,463959045,二姐品牌女鞋,306358673045,自建,0,精品女鞋5,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,单鞋,1356,13,7,2,0,0,4,0.009587,0.028822
2,20210131,463959045,二姐品牌女鞋,508583512045,自建,0,精品鞋子3,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,休闲鞋,2149,17,4,3,0,0,10,0.007911,0.028822
3,20210131,463959045,二姐品牌女鞋,508573587045,自建,0,精品女鞋1,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,高跟鞋,1509,14,3,3,0,0,8,0.009278,0.028822
4,20210131,463959045,二姐品牌女鞋,508579104045,自建,0,精品女鞋2,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,单鞋,1569,14,2,2,1,1,8,0.008923,0.028822
5,20210131,463959045,二姐品牌女鞋,508598209045,自建,0,精品鞋子6,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,高跟鞋,506,7,4,1,0,0,2,0.013834,0.028822
6,20210131,492537910,翠亮羊绒1987,348569261910,自建,0,799款,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,毛呢外套,876,11,5,2,0,2,2,0.012557,0.028822
7,20210131,492537910,翠亮羊绒1987,681616513910,自建,0,2199款,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,毛呢外套,1806,12,7,4,0,0,1,0.006645,0.028822
8,20210131,533306960,牛皮雪地靴 工厂,67416482960,自建,0,牛皮雪地靴5854款,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,雪地靴,1318,10,2,2,0,0,6,0.007587,0.028822
9,20210131,537058925,花姐教穿搭,712352078925,自建,0,8858福字旗袍裙,https://app.kwaixiaodian.com/merchant/shop/det...,1,1,...,连衣裙,392,6,3,1,0,0,2,0.015306,0.028822


In [9]:
#非分销库规则筛选下架商品
data_no_preoff=data_no_all[(data_no_all["quality_refound_rate"]>data_no_all["十倍一级类目品退率"])]
data_no_preoff.columns
len(data_no_preoff)
#各类目待下架商品数目计算
data_no_3=data_no_preoff.groupby("category_level_1_name")["item_id"].count()
data_no_3_1=pd.DataFrame(data_no_3)
data_no_3_1.columns=['非分销库待下架商品数'] #修改列索引名称
#数据合并
data_no_4=pd.merge(left=data_no_2,right=data_no_3_1,left_on="category_level_1_name",right_on="category_level_1_name")
data_no_print=data_no_4[["category_level_1_name","一级类目品退率","十倍一级类目品退率","非分销库待下架商品数","非分销库在架商品数"]]

### （2）食品相关二级类目计算

In [10]:
df_food_rate=data[(data["category_level_1_name"]=="食品")&(data["category_level_2_name"]=="生鲜")]
df_food_rate.columns
df_food_rate1=df_food_rate.groupby("category_level_2_name").agg(order_count=("order_count", "sum"),quality_refund_count=("quality_refund_count","sum"))
df_food_rate1
df_food_rate1.loc[:,"生鲜类目品退率"]=df_food_rate1["quality_refund_count"]/df_food_rate1["order_count"]
df_food_rate1=df_food_rate1.reset_index()
df_food_rate1.loc[:,"十倍一级类目品退率"]=df_food_rate1["生鲜类目品退率"].apply(lambda x:x*10)

In [11]:
data_fresh=pd.read_csv(path)
condition_fresh=(data_fresh['is_cps']==0)&(data_fresh['on_offline_status']==1)&(data_fresh['quality_refound_cnt']<=data_fresh['order_cnt'])&(data_fresh['category_level_1_name']=="食品")&(data_fresh['category_level_2_name']=="生鲜")
data_fresh=data_fresh[condition_fresh]                                                                                                                             

In [12]:
data_fresh_1=data_fresh.groupby("category_level_2_name")["item_id"].count()
data_fresh_1_1=pd.DataFrame(data_fresh_1)
data_fresh_1_1.columns=['非分销库在架商品数'] #修改列索引名称
data_fresh_1_1=data_fresh_1_1.reset_index()
data_fresh_2=pd.merge(left=data_fresh_1_1,right=df_food_rate1,left_on="category_level_2_name",right_on="category_level_2_name")
data_fresh_2

,category_level_2_name,非分销库在架商品数,order_count,quality_refund_count,生鲜类目品退率,十倍一级类目品退率
0,生鲜,223,1.035586e+09,5723692,0.005527,0.05527


In [13]:
data_fresh_ten=data_fresh_2[["category_level_2_name","十倍一级类目品退率"]]
data_fresh_all=pd.merge(left=data_fresh,right=data_fresh_ten,left_on="category_level_2_name",right_on="category_level_2_name",)
#data_fresh_all.head(10)
#规则计算
data_fresh_preoff=data_fresh_all[(data_fresh_all["quality_refound_rate"]>data_fresh_all["十倍一级类目品退率"])]
#将groupby格式转换为dataframe，待下架商品计算
data_fresh_3=data_fresh_preoff.groupby("category_level_2_name")["item_id"].count()
data_fresh_3_1=pd.DataFrame(data_fresh_3)
data_fresh_3_1.columns=['非分销库待下架商品数'] #修改列索引名称
data_fresh_3_1
#生鲜下架情况统计
data_fresh_4=pd.merge(left=data_fresh_2,right=data_fresh_3_1,left_on="category_level_2_name",right_on="category_level_2_name")
data_fresh_print=data_fresh_4[["category_level_2_name","生鲜类目品退率","十倍一级类目品退率","非分销库待下架商品数","非分销库在架商品数"]]
data_fresh_4.head()

,category_level_2_name,非分销库在架商品数,order_count,quality_refund_count,生鲜类目品退率,十倍一级类目品退率,非分销库待下架商品数
0,生鲜,223,1.035586e+09,5723692,0.005527,0.05527,4


### 非生鲜类目

In [14]:
#非生鲜类目规则计算
df_nofood_rate=data[(data["category_level_1_name"]=="食品")&(data["category_level_2_name"]!="生鲜")]
df_nofood_rate.columns
df_nofood_rate1=df_nofood_rate.groupby("category_level_1_name").agg(order_count=("order_count", "sum"),quality_refund_count=("quality_refund_count","sum"))
df_nofood_rate1
df_nofood_rate1.loc[:,"非生鲜类目品退率"]=df_nofood_rate1["quality_refund_count"]/df_nofood_rate1["order_count"]
df_nofood_rate1=df_nofood_rate1.reset_index()
df_nofood_rate1.loc[:,"十倍一级类目品退率"]=df_nofood_rate1["非生鲜类目品退率"].apply(lambda x:x*10)

In [15]:
#一级类目为食品，二级类目非生鲜商品
data_nofresh=pd.read_csv(path)
condition_nofresh=(data_nofresh['is_cps']==0)&(data_nofresh['on_offline_status']==1)&(data_nofresh['quality_refound_cnt']<=data_nofresh['order_cnt'])&(data_nofresh['category_level_1_name']=="食品")&(data_nofresh['category_level_2_name']!="生鲜")
data_nofresh=data_nofresh[condition_nofresh]   

In [16]:
data_nofresh_1=data_nofresh.groupby("category_level_1_name")["item_id"].count()
data_nofresh_1_1=pd.DataFrame(data_nofresh_1)
data_nofresh_1_1.columns=['非分销库在架商品数'] #修改列索引名称
data_nofresh_1_1=data_nofresh_1_1.reset_index()
data_nofresh_2=pd.merge(left=data_nofresh_1_1,right=df_nofood_rate1,left_on="category_level_1_name",right_on="category_level_1_name")
#data_fresh_2.loc[:,"十倍一级类目品退率"]=data_fresh_2["一级类目品退率"].apply(lambda x:x*10)
data_nofresh_2

,category_level_1_name,非分销库在架商品数,order_count,quality_refund_count,非生鲜类目品退率,十倍一级类目品退率
0,食品,954,1.239521e+10,20271239,0.001635,0.016354


In [17]:
data_nofresh_ten=data_nofresh_2[["category_level_1_name","十倍一级类目品退率"]]
data_nofresh_all=pd.merge(left=data_nofresh,right=data_nofresh_ten,left_on="category_level_1_name",right_on="category_level_1_name",)
data_nofresh_preoff=data_nofresh_all[(data_nofresh_all["quality_refound_rate"]>data_nofresh_all["十倍一级类目品退率"])]

data_nofresh_3=data_nofresh_preoff.groupby("category_level_1_name")["item_id"].count()
data_nofresh_3_1=pd.DataFrame(data_nofresh_3)
data_nofresh_3_1.columns=['非分销库待下架商品数'] #修改列索引名称

In [18]:
data_nofresh_4=pd.merge(left=data_nofresh_2,right=data_nofresh_3_1,left_on="category_level_1_name",right_on="category_level_1_name")
data_nofresh_print=data_nofresh_4[["category_level_1_name","非生鲜类目品退率","十倍一级类目品退率","非分销库待下架商品数","非分销库在架商品数"]]

In [19]:
data_nofresh_print

,category_level_1_name,非生鲜类目品退率,十倍一级类目品退率,非分销库待下架商品数,非分销库在架商品数
0,食品,0.001635,0.016354,53,954


### 数据合并

In [20]:
#数据合并
data_print=pd.concat([data_no_preoff,data_fresh_preoff,data_nofresh_preoff])
len(data_print)
#将生鲜类目名改一致，便于合并
data_nofresh_print.rename(columns={'非生鲜类目品退率': '一级类目品退率'}, inplace=True) 
data_fresh_print.rename(columns={'生鲜类目品退率': '一级类目品退率','category_level_2_name': 'category_level_1_name'}, inplace=True) 
#df.rename(columns=('$a': 'a', '$b': 'b', '$c': 'c', '$d': 'd', '$e': 'e'}, inplace=True) 
data_no_print=data_no_print.append(data_nofresh_print,ignore_index=True)
data_no_print=data_no_print.append(data_fresh_print,ignore_index=True)
data_no_print

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,category_level_1_name,一级类目品退率,十倍一级类目品退率,非分销库待下架商品数,非分销库在架商品数
0,图书文娱,0.000613,0.006135,1,5
1,孕婴/乐器/玩具,0.001939,0.019394,4,36
2,家居生活,0.002454,0.024541,194,849
3,家用电器,0.007848,0.078478,10,226
4,手机/数码/电脑办公,0.007219,0.072188,6,159
5,服饰/鞋靴/箱包,0.002882,0.028822,231,2270
6,汽车用品,0.006426,0.064259,5,31
7,珠宝/钟表,0.003941,0.039407,73,324
8,美妆/个护清洁,0.001292,0.012923,53,681
9,自营新业务,0.004404,0.044044,2,8


### 非分销库数据导出

In [21]:
import pandas as pd
writer = pd.ExcelWriter("/Users/philyu/kim/Kim file/2021-02/02品退数据/0131_非分销库品退率情况_分析.xlsx")
  
#data_is.to_excel(writer, sheet_name='分销库数据',index=None)
data_no1.to_excel(writer, sheet_name='非分销库数据',index=None)
#data_is_print.to_excel(writer, sheet_name='分销库品退率情况',index=None)
#data_is_preoff.to_excel(writer, sheet_name='分销库待下架商品名单')
data_no_print.to_excel(writer, sheet_name='非分销库品退率情况',index=None)
data_print.to_excel(writer, sheet_name='非分销库待下架商品名单',index=None)
#data_fresh_preoff.to_excel(writer, sheet_name='生鲜类目待下架商品名单',index=None)
writer.save()
writer.close()

/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### 去除下架商品外商品

In [17]:
data_is_on=data_is_all[(data_is_all["quality_refound_rate"]<=data_is_all["十倍三级类目品退率"])]
data_no_on=data_no_all[(data_no_all["quality_refound_rate"]<=data_no_all["十倍一级类目品退率"])]
data_fresh_on=data_fresh_all[(data_fresh_all["quality_refound_rate"]<=data_fresh_all["十倍一级类目品退率"])]
data_nofresh_on=data_nofresh_all[(data_nofresh_all["quality_refound_rate"]<=data_nofresh_all["十倍一级类目品退率"])]
data=pd.concat([data_no_on,data_is_on,data_fresh_on,data_nofresh_on])

In [18]:
data.to_csv('第二次任务下发_Result.csv')

In [19]:
data['item_id']

0      610127836549
1      660447217692
2      594427261918
3      624981335425
4      707492204265
           ...     
958    315611010171
959    666677808299
960     32732508156
961     32731084156
962      7502236156
Name: item_id, Length: 7807, dtype: int64